In [ ]:
# crew_ecommerce_3_agents.py
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

llm_arch = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
llm_dev  = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
llm_ui   = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

architect = Agent(
    role="Solution Architect",
    goal="Design a simple ecommerce system with catalog + order status, with clean APIs and data model.",
    backstory="You design pragmatic APIs and simple schemas for quick delivery.",
    llm=llm_arch,
    verbose=True
)

backend = Agent(
    role="Backend Engineer",
    goal="Implement the designed APIs using FastAPI with in-memory storage (no DB) and clear error handling.",
    backstory="You write clean Python services and provide runnable instructions.",
    llm=llm_dev,
    verbose=True
)

frontend = Agent(
    role="Frontend Engineer",
    goal="Build a simple React UI for catalog browsing + place order + view order status.",
    backstory="You build clean, minimal UIs with fetch calls and good UX.",
    llm=llm_ui,
    verbose=True
)

t1 = Task(
    description=(
        "Design the system.\n"
        "Requirements:\n"
        "- Catalog endpoints: list products, product detail\n"
        "- Orders: create order (productId, quantity), get order status by orderId\n"
        "- Use in-memory storage (no DB)\n"
        "- Provide: API spec, data models, example responses, and UI flow.\n"
        "Return output in Markdown with sections: Data Model, API Endpoints, UI Screens, Example Payloads.\n"
    ),
    agent=architect
)

t2 = Task(
    description=(
        "Implement the backend in FastAPI based on the architect spec.\n"
        "Must include:\n"
        "- app.py (single file)\n"
        "- In-memory product catalog (seed 5 items)\n"
        "- POST /orders creates an order and returns orderId + status\n"
        "- GET /orders/{orderId} returns status + items\n"
        "- GET /products and GET /products/{productId}\n"
        "- CORS enabled for localhost:3000\n"
        "- Clear instructions to run.\n"
        "Output ONLY the code for app.py in a code block, then a short run guide.\n"
    ),
    agent=backend
)

t3 = Task(
    description=(
        "Implement the frontend UI in React based on the architect spec and backend endpoints.\n"
        "Must include:\n"
        "- App.jsx (single file)\n"
        "- Product list + product detail\n"
        "- Quantity input and 'Place Order'\n"
        "- Show returned orderId\n"
        "- Order status lookup by orderId\n"
        "- Use fetch against http://localhost:8000\n"
        "- Minimal styling (plain CSS-in-JS or inline ok)\n"
        "Output ONLY the code for App.jsx in a code block, then a short run guide.\n"
    ),
    agent=frontend
)

crew = Crew(
    agents=[architect, backend, frontend],
    tasks=[t1, t2, t3],
    process=Process.sequential,
    verbose=True
)

if __name__ == "__main__":
    result = crew.kickoff()
    print("\n\n===== CREW OUTPUT =====\n")
    print(result)